In [2]:
import pandas as pd
import numpy as np
import os, sys
import model as md

In [2]:
# initial directory
zircolite_dir_path = "/kaggle/working/Zircolite"
dataset_path = "/kaggle/input/pwn-system/"
log_path = "/kaggle/working/test.log"

# On installe zircolite

In [3]:
!git clone https://github.com/wagga40/Zircolite.git

Cloning into 'Zircolite'...
remote: Enumerating objects: 2907, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 2907 (delta 176), reused 234 (delta 168), pack-reused 2651
Receiving objects: 100% (2907/2907), 41.23 MiB | 25.17 MiB/s, done.
Resolving deltas: 100% (881/881), done.


In [4]:
os.chdir(zircolite_dir_path)
!pip3 install -r /kaggle/working/Zircolite/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.1 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.3/385.3 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 16.2 MB/s eta 0:00:00


# On transforme les json dans le format adéquat pour le bon fonctionnement de zircolite qui est aplliqué sur chaque json

In [5]:
def remove_node(string):
    return "type" + string.split("type")[1]

df = pd.DataFrame()
for file in os.listdir(dataset_path):
    print(file)
    df_temp = pd.read_json(dataset_path+file,lines=True)
    df_linux_audit = df_temp[df_temp["sourcetype"] == "linux_audit"].reset_index().drop(axis=1, columns = ["index"])
    df_temp = df_linux_audit.copy()
    df_temp["raw"] = df_temp["raw"].apply(remove_node)
    
    print("log creation")
    df_temp.to_csv("test.log", header=None, index=False, sep=' ', mode='w')
    print("log created")
    
    print("start zircolite")
    !python3 zircolite.py --events /kaggle/working/Zircolite/test.log --ruleset rules/rules_linux.json --auditd
    print("end zircolite")
    
    df_linux_audit["tags"] = np.nan
    df_linux_audit["label"] = np.nan
    
    df_detected = pd.read_json("detected_events.json")
    df_detected = df_detected.dropna()
    for l in df_detected[["tags","matches"]].to_numpy():
        for m in l[1]:
            df_linux_audit["tags"][m["row_id"]]=l[0]
            df_linux_audit["label"][m["row_id"]]=1
    df_linux_audit["label"] = df_linux_audit["label"].fillna(0)
    
    df = pd.concat([df,df_linux_audit],axis=0)

pwnjutsu_dataset-system-json-n21.json
log creation
log created
start zircolite

    ███████╗██╗██████╗  ██████╗ ██████╗ ██╗     ██╗████████╗███████╗
    ╚══███╔╝██║██╔══██╗██╔════╝██╔═══██╗██║     ██║╚══██╔══╝██╔════╝
      ███╔╝ ██║██████╔╝██║     ██║   ██║██║     ██║   ██║   █████╗
     ███╔╝  ██║██╔══██╗██║     ██║   ██║██║     ██║   ██║   ██╔══╝
    ███████╗██║██║  ██║╚██████╗╚██████╔╝███████╗██║   ██║   ███████╗
    ╚══════╝╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝   ╚═╝   ╚══════╝
   -= Standalone SIGMA Detection tool for EVTX/Auditd/Sysmon Linux =-
    
[+] Checking prerequisites
[+] Extracting EVTX Using 'tmp-TT18UG2L' directory 
100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.21s/it]
[+] Processing EVTX
100%|█████████████████████████████████████████████| 1/1 [00:08<00:00,  8.99s/it]
[+] Creating model
[+] Inserting data
100%|████████████████████████████████| 148963/148963 [00:06<00:00, 21595.01it/s]
[+] Cleaning unused objects
[+] Loading ruleset from : 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


pwnjutsu_dataset-system-json-n33.json
log creation
log created
start zircolite

    ███████╗██╗██████╗  ██████╗ ██████╗ ██╗     ██╗████████╗███████╗
    ╚══███╔╝██║██╔══██╗██╔════╝██╔═══██╗██║     ██║╚══██╔══╝██╔════╝
      ███╔╝ ██║██████╔╝██║     ██║   ██║██║     ██║   ██║   █████╗
     ███╔╝  ██║██╔══██╗██║     ██║   ██║██║     ██║   ██║   ██╔══╝
    ███████╗██║██║  ██║╚██████╗╚██████╔╝███████╗██║   ██║   ███████╗
    ╚══════╝╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝   ╚═╝   ╚══════╝
   -= Standalone SIGMA Detection tool for EVTX/Auditd/Sysmon Linux =-
    
[+] Checking prerequisites
[+] Extracting EVTX Using 'tmp-AKTHXIWY' directory 
100%|█████████████████████████████████████████████| 1/1 [00:08<00:00,  8.62s/it]
[+] Processing EVTX
100%|█████████████████████████████████████████████| 1/1 [00:24<00:00, 24.60s/it]
[+] Creating model
[+] Inserting data
100%|████████████████████████████████| 420711/420711 [00:20<00:00, 20185.25it/s]
[+] Cleaning unused objects
[+] Loading ruleset from : 

In [6]:
df

,raw,sourcetype,source,time,host,tags,label
0,node=n21-vm3 type=PROCTITLE msg=audit(16211168...,linux_audit,/var/log/audit/audit.log,2021-05-15 22:14:29.925 UTC,n21-vm3,NaN,0.0
1,node=n21-vm3 type=SYSCALL msg=audit(1621116869...,linux_audit,/var/log/audit/audit.log,2021-05-15 22:14:29.925 UTC,n21-vm3,NaN,0.0
2,node=n21-vm3 type=PROCTITLE msg=audit(16211168...,linux_audit,/var/log/audit/audit.log,2021-05-15 22:14:29.925 UTC,n21-vm3,NaN,0.0
3,node=n21-vm3 type=SYSCALL msg=audit(1621116869...,linux_audit,/var/log/audit/audit.log,2021-05-15 22:14:29.925 UTC,n21-vm3,NaN,0.0
4,node=n21-vm3 type=PROCTITLE msg=audit(16211168...,linux_audit,/var/log/audit/audit.log,2021-05-15 22:14:29.925 UTC,n21-vm3,NaN,0.0
...,...,...,...,...,...,...,...
70596,node=n12-vm3 type=PATH msg=audit(1620555773.64...,linux_audit,/var/log/audit/audit.log,2021-05-09 08:02:00.000 UTC,n12-vm3,NaN,0.0
70597,node=n12-vm3 type=PATH msg=audit(1620555773.64...,linux_audit,/var/log/audit/audit.log,2021-05-09 08:02:00.000 UTC,n12-vm3,NaN,0.0
70598,node=n12-vm3 type=PATH msg=audit(1620555824.47...,linux_audit,/var/log/audit/audit.log,2021-05-09 00:19:00.000 UTC,n12-vm3,NaN,0.0
70599,node=n12-vm1 type=PATH msg=audit(1620594387.31...,linux_audit,/var/log/audit/audit.log,2021-05-09 00:10:00.000 UTC,n12-vm1,NaN,0.0


In [7]:
df.to_csv("labeled.csv")

In [8]:
print(df["tags"].isna().sum()/df_linux_audit.shape[0])

79.5056868882877


On remarque qu'il y a 79.5% de cellules vides

# raw expand
on parse la colonne raw et on crée le nombre de colonne nécessaire

In [2]:
import numpy as np

nrow = []

def raw_expand(df):
    rows_nb,_=df.shape

    for row in range(rows_nb):
        spl=df["raw"][row].split(' ') #on obtient la liste des col_name=value
        spl2=[elem.split("=") for elem in spl if len(elem.split("="))>1] # chaque element est une liste de forme [col_name, value]

        for cpl in spl2:
            if cpl[0] not in df.columns: #si la colonne n'a jamais été vue on la rajoute remplie de NaN
                df[cpl[0]]=np.nan
            df[cpl[0]][row]=cpl[1]
            
def detect_new_row(x):
    spl=x.split(' ')
    spl2=[elem.split("=") for elem in spl if len(elem.split("="))>1]
    for cpl in spl2:
        if cpl[0] not in nrow: #si la colonne n'a jamais été vue on la rajoute remplie de NaN
            nrow.append(cpl[0])

ici le jeu de données est rechargé apres avoir relancer le noyau pour éviter de dépasser la RAM

In [3]:
df_test = pd.read_csv("/kaggle/input/systemlabeled/labeled.csv")

In [4]:
df_test["raw"].apply(detect_new_row)

0          None
1          None
2          None
3          None
4          None
           ... 
6769593    None
6769594    None
6769595    None
6769596    None
6769597    None
Name: raw, Length: 6769598, dtype: object

In [5]:
print(nrow)

['node', 'type', 'msg', 'proctitle', 'arch', 'syscall', 'success', 'exit', 'a0', 'a1', 'a2', 'a3', 'items', 'ppid', 'pid', 'auid', 'uid', 'gid', 'euid', 'suid', 'fsuid', 'egid', 'sgid', 'fsgid', 'tty', 'ses', 'comm', 'exe', 'key', 'item', 'name', 'inode', 'dev', 'mode', 'ouid', 'ogid', 'rdev', 'nametype', 'cap_fp', 'cap_fi', 'cap_fe', 'cap_fver', 'cap_frootid', 'argc', 'hostname', 'addr', 'terminal', 'res', 'op', 'old', 'new', 'grantors', 'acct', 'old-auid', 'old-ses', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'saddr', 'cmd', 'fver', 'fp', 'fi', 'fe', 'old_pp', 'old_pi', 'old_pe', 'old_pa', 'pp', 'pi', 'pe', 'pa', 'frootid', 'sig', 'new_pp', 'new_pi', 'new_pe', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28', 'a29', 'a30', 'new-level', 'table', 'family', 'entries', 'list', 'audit_failure', 'audit_backlog_limit', 'path', 'reason', 'per', 'ver', 'format', 'kernel', 'subj', 'fd0', 'fd1', 'opid', 'oauid', 'oses',

on ajoute les colonnes manquantes

In [6]:
for n in nrow:
    df_test[n]=np.nan
nrow.append("raw")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [7]:
def fill_row(x):
    spl=x["raw"].split(' ') #on obtient la liste des col_name=value
    spl2=[elem.split("=") for elem in spl if len(elem.split("="))>1] # chaque element est une liste de forme [col_name, value]

    for cpl in spl2:
        x[cpl[0]]=cpl[1]
    return x

on remplie les colonnes pour 3 000 000 de lignes (séparer en plusieurs pour éviter tout problème avec la RAM

In [9]:
for i in range (1,31):
    end = df_test[(i-1)*100000:i*100000].apply(fill_row,axis=1)
    end.to_csv(f'parse{i}.csv')
    i=i+1

# cleaning

In [11]:
df = pd.read_csv("parse1.csv")
for i in range (2,31):
    df_temp = pd.read_csv(f"parse{i}.csv")
    df = pd.concat([df,df_temp],axis=0)
df.to_csv("all_parsed.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (7,12,13,15,17,18,19,20,33,35,36,37,53,54,55,56,57,60,61,64,65,66,67,68,69,70,71,72,73,74,75,80,82,84,86,90,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (7,12,13,15,17,18,19,20,33,35,36,37,53,54,55,56,57,60,61,64,65,66,67,68,69,70,71,72,73,74,75,80,82,84,86,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,111) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (7,12,13,15,17,18,19,20,33,35,36,37,53,54,55,56,57,60,61,64,65,66,67,68,69,70,71,72,73,74,80,82,84,86,90,9

In [3]:
df = pd.read_csv("/kaggle/working/all_parsed.csv")
df["tags"] = df["tags"].fillna(0)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (8,13,14,16,18,19,20,21,24,25,34,36,37,38,40,42,46,47,54,55,56,57,58,61,62,65,66,67,68,69,70,71,72,73,74,75,76,81,83,85,87,91,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,112,118,119,121,122,123,124,130) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
print("taux de valeur manquantes:\n")
na = df.isna().sum()/df.shape[0]
na

taux de valeur manquantes:



Unnamed: 0        0.0
Unnamed: 0.1      0.0
Unnamed: 0.1.1    0.0
raw               0.0
sourcetype        0.0
                 ... 
prom              1.0
old_prom          1.0
a2[1]             1.0
a2_len            1.0
a2[0]             1.0
Length: 138, dtype: float64

In [7]:
unusefull = []
for name,value in na.items():
    if value > 0.98:
        unusefull.append(name)
print(unusefull)
print("nombre colonne inutile : ",len(unusefull))

['grantors', 'acct', 'old-auid', 'old-ses', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'saddr', 'cmd', 'fver', 'fp', 'fi', 'fe', 'old_pp', 'old_pi', 'old_pe', 'old_pa', 'pp', 'pi', 'pe', 'pa', 'frootid', 'sig', 'new_pp', 'new_pi', 'new_pe', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28', 'a29', 'a30', 'new-level', 'table', 'family', 'entries', 'list', 'audit_failure', 'audit_backlog_limit', 'path', 'reason', 'ver', 'format', 'kernel', 'subj', 'fd0', 'fd1', 'opid', 'oauid', 'oses', 'ocomm', '\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\

In [10]:
df = df.drop(unusefull, axis=1)

In [12]:
df.to_csv("cleaned.csv")

# on lance les tests avec les différentes tailles d'ensemble d'entrainement et les différents modèles

In [ ]:
md.test()

# Demonstration

In [2]:
md.demo()

BENCHMARK START
Dataset loaded


###################################################
for 1000 rows


###################################################



for model XGBoost

confusion matrix
[[2800069    8248]
 [   5886  184797]]
true positive:  184797
false positive:  8248
true negative:  2800069
false negative:  5886
++++++++++++++++++++++++++++++++++++++++
Balanced data metrics:
precision: 0.9572742106762672
recall: 0.9691320149148063
tnr: 0.9970630096246257
accuracy: 0.9952870956985662
++++++++++++++++++++++++++++++++++++++++
Unbalanced data metrics:
f1_score: 0.9631666180210983
balanced_accuracy: 0.983097512269716
matthews correlation coefficient: 0.9606699797590342
Memory usage peak was 5520.358693MB
time elapsed : 144 sec



for xgboost categorical

confusion matrix
[[2799422     434]
 [  41884  148260]]
true positive:  148260
false positive:  434
true negative:  2799422
false negative:  41884
++++++++++++++++++++++++++++++++++++++++
Balanced data metrics:
precision: 0.99708125

# Exemple d'utilisation de la librairie

In [4]:
df = pd.read_csv("dataset/cleaned.csv",low_memory=False)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
print("is malware : ", md.is_malware(md.init_model("MLP",df),md.pretraitment(df)[0][1:3]))


confusion matrix
[[1961254    5392]
 [   4160  129194]]
true positive:  129194
false positive:  5392
true negative:  1961254
false negative:  4160
++++++++++++++++++++++++++++++++++++++++
Balanced data metrics:
precision: 0.9599363975450641
recall: 0.9688048352505362
tnr: 0.9972582762734117
accuracy: 0.9954514285714285
++++++++++++++++++++++++++++++++++++++++
Unbalanced data metrics:
f1_score: 0.9643502276629097
balanced_accuracy: 0.983031555761974
matthews correlation coefficient: 0.9619326231976215
is malware :  (array([0, 0], dtype=int64), [0.9989412211184951, 0.99847847509876])
